# GPELab simulation for Rabi coupled spin mixture BEC
I simulate the 2D dynamics for a cilindrical shape BEC spin mixture with Rabi coupling. <br>
The goal is to simulate the dynamics of the cloud during the adiabatic sweep of the detuning.


## ADIMENSIONALIZATION of GPE
We put $\hbar=1$ and $m = 1$.<br>
The two spin states are coupled by a Radio-frequency magnetic field with frequency $\Omega$. <br>
In 3D the interaction coupling is $g_{3D} = 4\pi a_s$.
Given the trap frequencies $\omega_r$ and $\omega_z$ we define the effective 2D interaction coupling constant 

$$g_{2D} = \frac{g_{3D}}{\sqrt{2\pi}a_z}= [E L^2],$$ 

where $a_z$ is the BEC characteristic size in the harmonic trap along the z direction. In general 

$$a_i = \sqrt{\frac{\hbar}{m\omega_i}}$$.  


To adimensionalize the BEC we have to define the characteristic dimensions for the system: time $T$, length $L$ and so the energy $E_0$. <br>
I start to take as choises $$T =1/\Omega; \quad L = 1/\sqrt{\omega_r}; \quad E_0 = \Omega$$ 
In that case the adimensional quantities are 
$$\Omega' = 1;\quad \omega_r ' = \omega_r/\Omega$$

$$a_z' = a_z / L = \sqrt{\omega_r/\omega_z}; \quad a_r' = a_r/L = 1$$

$$ g_{2D}' = \frac{\frac{2\sqrt{2\pi}a_s}{a_z}}{E_0 L^2}= \frac{g_{2D}}{\Omega \omega_r^{-1}}$$


In [15]:
import numpy as np

# Let's try to put some numbers. If for example we have:
wr = 200  #Hz
wz = 20   #Hz
W  = 5000 #Hz
ar = 1/np.sqrt(wr)
az = 1/np.sqrt(wz)

a0 = 52.917721e-11  # m Bohr radius
a11 = 30 * a0       # m
g3D = 4*np.pi*a11
g2D = (2*np.sqrt(2*np.pi)*a11)/az

# characteristic quantities are:
T = 1/W
L = np.sqrt(1/wr)
#L = np.sqrt(1/W)
E0 = W

#the adimensional quantities are:
W_A = W*T
wr_A = wr/(T**(-1))
wz_A = wz/(T**(-1))
ar_A = ar/L
az_A = az/L
g2D_A = g2D/(E0*L**2)

print("\nThe adimensional quantities are:\n")
print(f"W_A   = {W_A}" f"; wr_A  = {wr_A}" f"; wz_A  = {wz_A}")
print(f"ar_A  = {ar_A}" f"; az_A  = {az_A}")
print(f"g2D_A = {g2D_A}")


The adimensional quantities are:

W_A   = 1.0; wr_A  = 0.04; wz_A  = 0.004
ar_A  = 1.0; az_A  = 3.162277660168379
g2D_A = 2.3728268911743695e-08


With those choises for the characteristic lengths so the initial size of the BEC will be unitary, as well the Rabi frequency. The interaction coupling is small, but I believe it is reasonable since interactions in the BEC are usually weak.  
g2D_A can become a bit bigger (1e-7) if one takes $L = 1/\sqrt(\Omega)$. The problem in having $g_{2D}'$ is very small (or very large) is that numerical instabilities might arise in solving the Gross-Pitaevskii equation.

The weak point in taking these definitions for L,T and E0, is that is the if $\omega_r'$ becomes much smaller than 1, length scales in the simulation might become large, leading to increased computational costs.

# GPE adimensional coefficients
Given the equation 
$$i\hbar \partial_t \begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix} = \left[
-\frac{\hbar^2}{2m} \Delta + V_{ext} + \hbar/2 \begin{pmatrix} \delta & -\Omega \\ -\Omega & -\delta \end{pmatrix} +
\begin{pmatrix} g_{11}|\Psi_1|^2+g_{12}|\Psi_2|^2 & 0 \\ 0 & g_{22}|\Psi_2|^2+g_{12}|\Psi_1|^2 \end{pmatrix}
\begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix}
\right] $$

we can write the its adimensional version:
$$ i \partial_t \begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix} = \left[
-\alpha \Delta + V_{ext} +
1/2 \begin{pmatrix} \delta T & -\Omega T \\ -\Omega T & -\delta T \end{pmatrix} +
\beta \begin{pmatrix} \gamma_{11}|\Psi_1|^2+\gamma_{12}|\Psi_2|^2 & 0 \\ 0 & \gamma_{22}|\Psi_2|^2+\gamma_{12}|\Psi_1|^2 \end{pmatrix}
\begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix}
\right] $$

where
$$\alpha = \frac{1}{2} \frac{\hbar}{m} \frac{T}{L^2} \rightarrow  \frac{1}{2} \frac{T}{L^2} $$

$$\beta =  1; \quad \gamma_i = {g_{2D}}_i'$$

and of course also $\Psi$ is normalized.

In [17]:
# with our characteristic lengths:
a11 = 30*a0
a22 = 80*a0
a12 = -20*a0

g2D_11 = (2*np.sqrt(2*np.pi)*a11)/az
g2D_22 = (2*np.sqrt(2*np.pi)*a22)/az
g2D_12 = (2*np.sqrt(2*np.pi)*a12)/az

alpha = 0.5*(T/L**2)
beta = 1
gamma_11 = g2D_11/(E0*L**2)
gamma_22 = g2D_22/(E0*L**2)
gamma_12 = g2D_12/(E0*L**2)

print("\nThe adimensional GPE coefficients are:\n")
print(f"alpha   = {alpha}")
print(f"gamma_11  = {gamma_11}" f"; gamma_22  = {gamma_22}" f"; gamma_12  = {gamma_12}")


The adimensional GPE coefficients are:

alpha   = 0.02
gamma_11  = 1.423696134704622e-08; gamma_22  = 3.7965230258789917e-08; gamma_12  = -9.491307564697479e-09


## Ground state solution
### Initial guess for $\Psi$

I define the wavefunction $\Psi$ to use as initial guess like:
